In [33]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import pandas as pd
import matplotlib
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import altair as alt
import sys
import os
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

spot_diameter = 89.43 #pixels

In [57]:
# # Load gene_ids
# gene_ids = []
# with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
#     for line in file:
#         # Remove trailing newline characters and any leading/trailing whitespaces
#         line = line.strip()
#         gene_ids.append(line)

# # Load coordinates
# coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# # Load cell_barcode
# with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
#     cell_barcode = pickle.load(file)

# # Load connected_components (assume already hav)
# with open("/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl", 'rb') as file:
#     filtered_connected_components = pickle.load(file)

# # Load pathologist's label
# data_name = 'PDAC_64630'
# if data_name == 'PDAC_64630':
#     pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
#     pathologist_label=[]
#     with open(pathologist_label_file) as file:
#         csv_file = csv.reader(file, delimiter=",")
#         for line in csv_file:
#             pathologist_label.append(line)	
    	
#     barcode_type=dict() # record the type (annotation) of each spot (barcode)
#     for i in range (1, len(pathologist_label)):
#         barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

In [69]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_B1_gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_B1_coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_experiments/exp1_B1_cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)

# Load connected_components (assume already have it)
with open("/Users/victoriagao/local_docs/NEST/stored_variables/exp1_B1_filtered_connected_components.pkl", 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/exp1_B1_IX_annotation_pathology.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

In [70]:
# Make barcode_info
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
barcode_info.append(["cell_code", "x-coordinate", "y-coordinate", "component", "color", "barcode_number"]) # add title
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0, 0, 0, 0]) # last entries will hold the component number later
    i=i+1

In [71]:
filename_str = 'exp1_B1_NEST_combined_rank_product_output_PDAC_130355_B1_top20percent'+'.csv'
# filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
df = pd.read_csv(inputFile, sep=",")
csv_record_final = df.values.tolist()
df_column_names = list(df.columns)
csv_record_final = [df_column_names] + csv_record_final

component_list = dict()
for record_idx in range (1, len(csv_record_final)-1): #last entry is a dummy for histograms, so ignore it.
    record = csv_record_final[record_idx]
    i = record[6] # from id
    j = record[7] # to id
    component_label = record[5] # component
    barcode_info[i+1][3] = component_label 
    barcode_info[j+1][3] = component_label
    component_list[component_label] = ''

component_list[0] = ''
unique_component_count = len(component_list.keys())

In [42]:
csv_record_final

[['from_cell',
  'to_cell',
  'ligand',
  'receptor',
  'attention_score',
  'component',
  'from_id',
  'to_id'],
 ['ATTATACTTTGCTCGT-1',
  'TTAATCAGTACGTCAG-1',
  'TGFB1',
  'ITGB5',
  0.97862254,
  -1,
  327,
  1279],
 ['ATTCAGGACCTATTTC-1',
  'CTAGCCGATGTTATGA-1',
  'TGFB1',
  'ITGB6',
  0.97862312,
  -1,
  330,
  608],
 ['ATTATGCCATAGGGAG-1',
  'CTTGTACTTGTTGACT-1',
  'TGFB1',
  'ITGB5',
  0.978622361,
  -1,
  328,
  673],
 ['GCGGGAACCAGGCCCT-1',
  'CTTGTACTTGTTGACT-1',
  'TGFB1',
  'ITGB5',
  0.978622347,
  -1,
  811,
  673],
 ['ACGCCAGATGATTTCT-1',
  'TTAATCAGTACGTCAG-1',
  'TGFB1',
  'ITGB5',
  0.97862243,
  -1,
  123,
  1279],
 ['CGACTTTGTATAGCCT-1',
  'ACTTTACCCTCATGAA-1',
  'TGFB1',
  'ITGB5',
  0.978622476,
  -1,
  521,
  157],
 ['ATTCAGGACCTATTTC-1',
  'CGTTTCACTTCGGGCG-1',
  'TGFB1',
  'ITGB5',
  0.978622555,
  -1,
  330,
  592],
 ['GCTCTCGGGTACCGAA-1',
  'ATAGGTTGGGCAGATG-1',
  'TGFB1',
  'ITGB6',
  0.978622287,
  -1,
  835,
  262],
 ['ATTATACTTTGCTCGT-1',
  'CCTTCTTGATC

In [72]:
# Gather relavant and match with barcodes:
# From the csv_record_final:

from_df = pd.DataFrame(csv_record_final, columns=["from_cell", "to_cell", "ligand", "receptor", "attention_score", "component", "from_id", "to_id"])[["from_cell", "component", "from_id"]]
to_df = pd.DataFrame(csv_record_final, columns=["from_cell", "to_cell", "ligand", "receptor", "attention_score", "component", "from_id", "to_id"])[["to_cell", "component", "to_id"]]

from_df.columns = ["barcode", "component", "cell_id"]
to_df.columns = ["barcode", "component", "cell_id"]

combined_barcode_df = pd.concat([from_df, to_df], ignore_index=True)
# combined_barcode_df = combined_barcode_df.drop(0)

In [73]:
# Create a dictionary to map barcodes to sets
barcode_set_mapping = {}
for i, barcode_set in enumerate(filtered_connected_components, start=1):
    for barcode in barcode_set:
        barcode_set_mapping[barcode] = i

# Create a new 'set' column in combined_barcode_df
combined_barcode_df['region'] = combined_barcode_df['barcode'].map(barcode_set_mapping)

In [74]:
# Create a graph
# set1 = altairThemes.get_colour_scheme("Set1", id_label)
# colors = set1
# colors[0] = '#000000'
ids = []
x_index=[]
y_index=[]
# colors_point = []
for i in range (0, len(barcode_info)):    
    ids.append(i)
    x_index.append(barcode_info[i][1])
    y_index.append(barcode_info[i][2])
    # color_index = barcode_info[i][3]
    # print(colors[color_index])
    # colors_point.append(colors[color_index]) 

g = nx.MultiDiGraph(directed=True) #nx.Graph()
for i in range (1, len(barcode_info)-1):
    # label_str =  str(i)+'_c:'+str(barcode_info[i][3])+'_'
    # #if barcode_type[barcode_info[i][0]] == 'zero':
    # #    continue
    # # print(barcode_info[i][3])
    # if barcode_type[barcode_info[i][0]] == 0: #stroma
    #     marker_size = 'circle'
    #     label_str = label_str + 'stroma'
    #     print("label successful")
    # elif barcode_type[barcode_info[i][0]] == 1: #tumor
    #     marker_size = 'box'
    #     label_str = label_str + 'tumor'
    #     print(barcode_info[i][3],"successful")

    # else:
    #     marker_size = 'ellipse'
    #     label_str = label_str + 'acinar_reactive'
	
    # g.add_node(int(ids[i]), x=int(x_index[i]), y=int(y_index[i]), label = label_str, pos = str(x_index[i])+","+str(-y_index[i])+" !", physics=False, shape = marker_size, color=matplotlib.colors.rgb2hex(colors_point[i]))
    # g.add_node(int(ids[i]), x=int(x_index[i]), y=int(y_index[i]), label = label_str, pos = str(x_index[i])+","+str(-y_index[i])+" !", physics=False, shape = marker_size)
    g.add_node(int(ids[i]), x=int(x_index[i]), y=int(y_index[i]), pos = str(x_index[i])+","+str(-y_index[i])+" !", physics=False)

    # print(colors_point[i])

nt = Network( directed=True, height='1000px', width='100%')

for interaction in csv_record_final[1:]:
    ligand_id = interaction[6]
    receptor_id = interaction[7]
    ligand_name = interaction[2]
    receptor_name = interaction[3]
    edge_score = interaction[4]
    # print(type(interaction[4]))
    interaction_label = "L:"+ligand_name+", R:"+receptor_name

    g.add_edge(ligand_id, receptor_id, interaction_label=interaction_label, value=np.float64(edge_score))
    # g.add_edge(ligand_id, receptor_id, interaction_label=interaction_label, value=np.float64(edge_score), color=colors_point[ligand_id])

    # print(colors_point[ligand_id])


num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")

Number of nodes: 1478
Number of edges: 416801


In [75]:
barcode_info

[['cell_code',
  'x-coordinate',
  'y-coordinate',
  'component',
  'color',
  'barcode_number'],
 ['AAACAGCTTTCAGAAG-1', 9464, 13903, -1, 0, 0, 0],
 ['AAACCGGGTAGGTACC-1', 9283, 12018, -1, 0, 0, 0],
 ['AAACCGTTCGTCCAGG-1', 11004, 10622, -1, 0, 0, 0],
 ['AAACCTCATGAAGTTG-1', 8424, 12915, -1, 0, 0, 0],
 ['AAACGAGACGGTTGAT-1', 8053, 6963, -1, 0, 0, 0],
 ['AAACTCGTGATATAAG-1', 5967, 3598, -1, 0, 0, 0],
 ['AAACTGCTGGCTCCAA-1', 9785, 8146, -1, 0, 0, 0],
 ['AAAGACTGGGCGCTTT-1', 7044, 13318, -1, 0, 0, 0],
 ['AAAGCTTGCCTACATA-1', 6481, 2702, -1, 0, 0, 0],
 ['AAAGGGATGTAGCAAG-1', 6161, 8658, -1, 0, 0, 0],
 ['AAAGGGCAGCTTGAAT-1', 6176, 12230, -1, 0, 0, 0],
 ['AAAGTCGACCCTCAGT-1', 8425, 13312, -1, 0, 0, 0],
 ['AAATACCTATAAGCAT-1', 10129, 7946, -1, 0, 0, 0],
 ['AAATAGCTTAGACTTT-1', 8214, 4283, -1, 0, 0, 0],
 ['AAATGATTCGATCAGC-1', 5622, 3798, -1, 0, 0, 0],
 ['AAATGGTCAATGTGCC-1', 7719, 9743, -1, 0, 0, 0],
 ['AAATTAACGGGTAGCT-1', 7889, 9047, -1, 0, 0, 0],
 ['AAATTAATAAGCGCGA-1', 7567, 14407, -1, 0,

In [81]:
# Based on Leiden clustering
# Get unique regions from the 'region' column in combined_barcode_df
unique_regions = sorted(combined_barcode_df['region'].unique())

result_data = []
# Initialize an empty list to store modularity values
# modularity_values = []

# Iterate through each unique region
for region in unique_regions:
    # Create a subgraph for the current region
    region_subset_graph = g.subgraph(combined_barcode_df[combined_barcode_df['region'] == region]['cell_id'])
    
    if len(region_subset_graph.edges()) > 0:
        if bipartite.is_bipartite(region_subset_graph) == False:
            # Perform community detection (Leiden algorithm)
            leiden_comm = algorithms.leiden(region_subset_graph)
            set_communities_list = leiden_comm.communities
            
            # Calculate modularity for the region's subgraph and communities
            modularity = nx.community.modularity(region_subset_graph, set_communities_list)
            
            print("modularity for region", str(region),"is", modularity)
            result_data.append({'# of community': len(set_communities_list), f"modularity of region{region}": modularity})
            # Append the modularity value to the list
            # modularity_values.append(modularity)
        else:
            print("the graph is disconnected for region", str(region))
    else:
        print("The graph is empty for region", str(region))

    results_df_leiden = pd.DataFrame(result_data)
    # Save
    table_path = os.path.join("/Users/victoriagao/local_docs/NEST/output/", "leiden_modularity_regions.csv")
    results_df_leiden.to_csv(table_path, index=False)

# The 'modularity_values' list now contains modularity values for each unique region



The graph is empty for region nan
modularity for region 1.0 is 0.0
modularity for region 2.0 is 0.6671133188653284
modularity for region 3.0 is 0.2055749224521582
modularity for region 4.0 is 0.10809613601031817
modularity for region 5.0 is 0.004620565431905366
modularity for region 6.0 is 1.1102230246251565e-16
modularity for region 7.0 is 0.03649021977409858
modularity for region 8.0 is 0.31490177742486575
modularity for region 9.0 is 0.23665523516596182
modularity for region 10.0 is 0.3574346286872112
modularity for region 11.0 is 0.017077551020408185
modularity for region 12.0 is 0.0
modularity for region 13.0 is 0.1738436419966913
modularity for region 14.0 is 0.1981230835480254
the graph is disconnected for region 15.0
modularity for region 16.0 is 0.0
modularity for region 17.0 is 0.034946564729659374
modularity for region 18.0 is 0.0
modularity for region 19.0 is 0.0
modularity for region 20.0 is 0.0
modularity for region 21.0 is 0.004861044920412283
modularity for region 22.0 

In [79]:
# Based on Louvain clustering
# Get unique regions from the 'region' column in combined_barcode_df
unique_regions = sorted(combined_barcode_df['region'].unique())

# Initialize an empty list to store modularity values
result_data = []

# Iterate through each unique region
for region in unique_regions:
    # Create a subgraph for the current region
    region_subset_graph = g.subgraph(combined_barcode_df[combined_barcode_df['region'] == region]['cell_id'])
    
    if len(region_subset_graph.edges()) > 0:
        if bipartite.is_bipartite(region_subset_graph) == False:
            # Perform community detection (Leiden algorithm)
            louvain_comm = nx.community.louvain_communities(region_subset_graph)
            
            # Calculate modularity for the region's subgraph and communities
            modularity = nx.community.modularity(region_subset_graph, louvain_comm)
            
            print("modularity for region", str(region),"is", modularity)
            # print("there are",len(louvain_comm),"communities")
            result_data.append({'# of community': len(louvain_comm), f"modularity region{region}": modularity})
            # Append the modularity value to the list
            # modularity_values.append(modularity)
        else:
            print("the graph is disconnected for region", str(region))
    else:
        print("The graph is empty for region", str(region))

    results_df_louvain = pd.DataFrame(result_data)
    # Save
    table_path = os.path.join("/Users/victoriagao/local_docs/NEST/output/", "louvain_modularity_regions.csv")
    results_df_louvain.to_csv(table_path, index=False)

# The 'modularity_values' list now contains modularity values for each unique region




The graph is empty for region nan
modularity for region 1.0 is 0.0
modularity for region 2.0 is 0.6393093779305151
modularity for region 3.0 is 0.1954830394755394
modularity for region 4.0 is 0.10809613601031817
modularity for region 5.0 is 0.004620565431905366
modularity for region 6.0 is -0.013602120535714239
modularity for region 7.0 is 0.03649021977409858
modularity for region 8.0 is 0.31490177742486575
modularity for region 9.0 is 0.23665523516596182
modularity for region 10.0 is 0.3574346286872112
modularity for region 11.0 is 0.002546938775510219
modularity for region 12.0 is 0.0
modularity for region 13.0 is 0.1738436419966913
modularity for region 14.0 is 0.1981230835480254
the graph is disconnected for region 15.0
modularity for region 16.0 is 0.0
modularity for region 17.0 is 0.034946564729659374
modularity for region 18.0 is -0.03265306122448988
modularity for region 19.0 is 0.0
modularity for region 20.0 is 0.0
modularity for region 21.0 is 0.004861044920412283
modularity 

In [82]:
results_df_louvain

,# of community,modularity region1.0,modularity region2.0,modularity region3.0,modularity region4.0,modularity region5.0,modularity region6.0,modularity region7.0,modularity region8.0,modularity region9.0,...,modularity region35.0,modularity region36.0,modularity region37.0,modularity region38.0,modularity region39.0,modularity region40.0,modularity region41.0,modularity region42.0,modularity region43.0,modularity region44.0
0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NaN,0.639309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,0.195483,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,NaN,0.108096,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,NaN,NaN,NaN,NaN,0.004621,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,NaN,NaN,NaN,NaN,NaN,-0.013602,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,NaN,NaN,NaN,NaN,NaN,NaN,0.03649,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.314902,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.236655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Based on component number
# target_region_df = combined_barcode_df[combined_barcode_df['region']==1]

# grouped = target_region_df.groupby('component')['cell_id'].apply(set).reset_index()
# list_of_sets = grouped['cell_id'].tolist()
# len(list_of_sets)


unique_regions = sorted(combined_barcode_df['region'].unique())

result_data = []

# Iterate through each unique region
for region in unique_regions:
    # Create a subgraph for the current region
    target_region_df = combined_barcode_df[combined_barcode_df['region']==region]
    grouped = target_region_df.groupby('component')['cell_id'].apply(set).reset_index()
    list_of_sets = grouped['cell_id'].tolist()
    print("# of communities based on component assignments for region",region,"is:",len(list_of_sets))
    result_data.append({'# of community': len(list_of_sets)})

results_df_components = pd.DataFrame(result_data)
    # Save
file_name = f"BIG_log_odds.csv"
table_path = os.path.join("/Users/victoriagao/local_docs/NEST/output/", "numofcommunities_by_component.csv")
results_df_components.to_csv(table_path, index=False)



# of communities based on component assignments for region nan is: 0
# of communities based on component assignments for region 1.0 is: 1
# of communities based on component assignments for region 2.0 is: 1
# of communities based on component assignments for region 3.0 is: 1
# of communities based on component assignments for region 4.0 is: 1
# of communities based on component assignments for region 5.0 is: 1
# of communities based on component assignments for region 6.0 is: 1
# of communities based on component assignments for region 7.0 is: 1
# of communities based on component assignments for region 8.0 is: 1
# of communities based on component assignments for region 9.0 is: 1
# of communities based on component assignments for region 10.0 is: 1
# of communities based on component assignments for region 11.0 is: 1
# of communities based on component assignments for region 12.0 is: 1
# of communities based on component assignments for region 13.0 is: 1
# of communities based on comp

In [14]:
unique_regions

array([nan,  1.,  3.,  2.,  5.,  7., 16., 11.,  4.,  6.,  9.,  8., 13.,
       20., 14., 18., 15., 17.])